<a href="https://colab.research.google.com/github/CaterinaBi/sentiment-analysis-pipeline/blob/main/sentiment_analysis_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sentiment analysis pipeline in TensorFlow.

In [1]:
import numpy as np 
import pandas as pd # data processing
import tensorflow as tf
from tensorflow import keras
from collections import Counter

Dataset (Stanford Treebank)

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive', force_remount=True)
# %cd /gdrive

In [2]:
# stanford_df = pd.read_csv('/gdrive/MyDrive/datasets/dataset.csv', on_bad_lines='skip')
stanford_df = pd.read_csv('dataset.csv', on_bad_lines='skip')

In [3]:
print(len(stanford_df))

239232


In [4]:
train_df = stanford_df[:120000]
test_df = stanford_df[120001:]
# no valid set ??

train_df.head()

,snippet,score
0,!,0.50000
1,',0.50000
2,' (,0.44444
3,' ( the cockettes,0.50000
4,' ( the cockettes ),0.42708


Text preprocessing

In [5]:
train = tf.data.Dataset.from_tensor_slices((train_df['snippet'],train_df['score']))
test = tf.data.Dataset.from_tensor_slices((test_df['snippet'],test_df['score']))

In [6]:
# Preprocessing 

def preprocess(X_batch, y_batch):
    # Truncate to first 300 characters of each review
    X_batch = tf.strings.substr(X_batch, 0, 300)
    # replace brackets with spaces
    X_batch = tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ")
    # regex for replacing everything except characters and quotes with spaces
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    # Split the string to individual words
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [7]:
# Create vocabulary of words

vocabulary = Counter()
for X_batch, y_batch in train.batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [8]:
vocabulary.most_common()[:3]

[(b'<pad>', 2061802), (b'the', 36954), (b'and', 28200)]

In [9]:
# restricting to the most common 10000 words

vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]]

In [10]:
# mapping words to ids
# oov buckets: randomly map new words to one of the oov buckets

words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [11]:
# final encodings

def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = train.batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

Model

In [ ]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1)
])
model.compile(loss="mean_squared_error", optimizer="adam",
              #metrics=["accuracy"]
             )
history = model.fit(train_set, epochs=20)

Epoch 1/20
 771/3750 [=====>........................] - ETA: 1:58 - loss: 0.0266